## **Resultado final - "Emergencias UNGRD"**
----
Este notebook integra los dos CSVs previos en uno solo. Se realizó un profundo proceso de limpieza en, aproximadamente, 18 fuentes de datos (Excel y JSON) que derivo en el CSV final que resulta del procesamiento de datos realizado en este notebook.

**Elaborado con base en**
* [​​​​​​​​​​​​Consolidado anual de emergencias ​​| UNGRD](https://portal.gestiondelriesgo.gov.co/Paginas/Consolidado-Atencion-de-Emergencias.aspx)
* [Emergencias UNGRD | Datos Abiertos Colombia](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Emergencias-UNGRD-/wwkg-r6te/about_data)

#### ***Importación de paquetes***

In [1]:
import pandas as pd

#### ***Cargar los CSVs mediante Pandas***

In [2]:
emergenciasConsolidados = pd.read_csv('../CSVs/EmergenciasCC(2003-2018, 2023)_RegionPacifico.csv')
emergenciasDatosAbiertos = pd.read_csv('../CSVs/EmergenciasCC(2019-2022)_RegionPacifico.csv')

#### ***Imprimir una muestra de los datos obtenidos***

##### **Obtenido desde los consolidados *(sin filtrar aún por evento)***

In [3]:
emergenciasConsolidados.sample(5)

,AÑO,MES,FECHA,EVENTO,MUNICIPIO,DEPARTAMENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
8896,2010.0,11.0,2010-11-14 00:00:00,INUNDACION,MAHATES,BOLIVAR,0,0,0,2600,520,0,0,0.0
40727,2023.0,2.0,2023-02-28,INCENDIO FORESTAL,SALAMINA,MAGDALENA,0,0,0,0,0,0,0,1.0
17220,2013.0,1.0,2013-01-21,INCENDIO FORESTAL,VALLE DE SAN JOSE,SANTANDER,0,0,0,0,0,0,0,3.0
44067,2023.0,12.0,2023-12-22,INCENDIO ESTRUCTURAL,BELLO,ANTIOQUIA,0,2,0,0,0,6,0,0.0
8111,2010.0,7.0,2010-07-28 00:00:00,VENDAVAL,QUIMBAYA,QUINDIO,0,0,0,10,2,0,0,0.0


##### **Obtenido desde el dataset de Datos Abiertos**

In [4]:
emergenciasDatosAbiertos.sample(5)

,AÑO,MES,FECHA,DEPARTAMENTO,MUNICIPIO,EVENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
3552,2022,12,2022-12-29,NARIÑO,CORDOBA,INCENDIO DE COBERTURA VEGETAL,0,0,0,0,0,0,0,1.0
1842,2022,9,2022-09-08,CAUCA,PIENDAMO,INCENDIO DE COBERTURA VEGETAL,0,0,0,0,0,0,0,2.0
2551,2022,3,2022-03-01,NARIÑO,SAPUYES,MOVIMIENTO EN MASA,0,0,0,0,0,0,3,0.0
2003,2022,11,2022-11-10,VALLE DEL CAUCA,PRADERA,CRECIENTE SUBITA,0,0,0,70,14,0,0,0.0
630,2020,4,2020-04-23,VALLE DEL CAUCA,RESTREPO,TEMPORAL,0,0,0,15,3,0,0,0.0


#### ***Concatenar los dos dataframes***

In [5]:
# Lista de nombres de los dataframes
dataframes = [emergenciasConsolidados, emergenciasDatosAbiertos]

# Concatenar los dataframes en uno solo
emergencias = pd.concat(dataframes)

emergencias.sample(5)

,AÑO,MES,FECHA,EVENTO,MUNICIPIO,DEPARTAMENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
3690,2007.0,5.0,2007-05-24,INUNDACION,MAJAGUAL,SUCRE,1,0,0,23225,4645,0,0,0.0
22346,2014.0,5.0,2014-05-12 00:00:00,DESLIZAMIENTO,LEBRIJA,SANTANDER,0,0,0,5,1,0,0,0.0
32404,2017.0,1.0,2017-01-30,CONTAMINACION,EL BANCO,MAGDALENA,0,0,0,0,0,0,0,0.0
16440,2012.0,10.0,2012-10-07,INCENDIO FORESTAL,CALI,VALLE DEL CAUCA,0,0,0,0,0,0,0,0.0
2288,2005.0,12.0,2005-12-01,INUNDACION,LA VIRGINIA,RISARALDA,0,0,0,200,40,40,0,0.0


## **Limpieza de datos**
---

### ***Consolidados - UNGRD***

#### ***Escogemos las filas con los eventos de nuestro interés***
En especial aquellos que son impactados fuertemente por el cambio climático.

In [6]:
emergencias["EVENTO"] = emergencias["EVENTO"].fillna("")

emergencias["EVENTO"] = emergencias["EVENTO"].str.upper()

emergencias["EVENTO"] = emergencias["EVENTO"].str.strip()

# Diccionario de reemplazo
diccionario_reemplazos = {
    "INUNDACION": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "INUNDACIÓN": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "AVENIDA TORRENCIAL": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "CRECIENTE SUBITA": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "CRECIENTE SÚBITA": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "CRECIENTE": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "LLUVIAS": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "VENDAVAL-INCENDIO ESTRUCTURAL": "VENDAVAL",
    "DELIZAMIENTO": "DESLIZAMIENTO",
    "MOVIMIENTO EN MASA": "DESLIZAMIENTO",
    "REMOCIÓN EN MASA": "DESLIZAMIENTO",
    "REMOCION EN MASA": "DESLIZAMIENTO",
    "MAREJADA": "MAREJADAS/MAR DE LEVA",
    "MAREJADAS": "MAREJADAS/MAR DE LEVA",
    "MAR DE LEVA": "MAREJADAS/MAR DE LEVA",
    "MAR DELEVA": "MAREJADAS/MAR DE LEVA",
    "EROSION-INCENDIO ESTRUCTURAL": "EROSION",
    "EROSION COSTERA": "EROSION",
    "FENOMENO ATMOSFERICO": "TEMPORAL",
    "CICLON TROPICAL: DEPRESION/TORMENTA/HURACAN": "DEPRESION/TORMENTA/HURACAN",
    "CICLÓN, TROPICAL, DEPRESIÓN/TORMENTA/HURACÁN": "DEPRESION/TORMENTA/HURACAN",
    "TORMENTA ELECTRICA": "DEPRESION/TORMENTA/HURACAN",
    "TORNADO": "DEPRESION/TORMENTA/HURACAN",
    "DEPRESION TROPICAL": "DEPRESION/TORMENTA/HURACAN",
    "HURACAN": "DEPRESION/TORMENTA/HURACAN",
}

# Aplicar los cambios
emergencias["EVENTO"] = emergencias["EVENTO"].replace(diccionario_reemplazos)

In [7]:
# Lista de eventos a filtrar (ajustada a los nombres proporcionados)
eventos_a_filtrar = [
    "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "INCENDIO FORESTAL",
    "DESLIZAMIENTO",
    "VENDAVAL",
    "SEQUIA",
    "DEPRESION/TORMENTA/HURACAN",
    "GRANIZADA",
    "TEMPORAL",
    "EROSION",
    "HELADA",
    "MAREJADAS/MAR DE LEVA"
]

# Filtrar el DataFrame
emergencias = emergencias[emergencias["EVENTO"].isin(eventos_a_filtrar)]

# Mostrar el DataFrame filtrado
emergencias

,AÑO,MES,FECHA,EVENTO,MUNICIPIO,DEPARTAMENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
0,2003.0,1.0,2003-01-01,AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA,BAGADO,CHOCO,1,0,0,700,175,0,0,0.0
1,2003.0,1.0,2003-01-01,AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA,MEDIO ATRATO,CHOCO,0,0,0,3161,691,0,0,0.0
2,2003.0,1.0,2003-01-01,VENDAVAL,LITORAL DEL SAN JUAN,CHOCO,0,0,0,995,199,0,0,0.0
3,2003.0,1.0,2003-01-03,INCENDIO FORESTAL,LA TEBAIDA,QUINDIO,0,0,0,0,0,0,0,0.0
4,2003.0,1.0,2003-01-03,AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA,LLORO,CHOCO,0,0,0,1076,269,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3544,2022.0,12.0,2022-12-26,AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA,ROSAS,CAUCA,0,0,0,0,0,0,0,0.0
3545,2022.0,12.0,2022-12-26,DESLIZAMIENTO,ROSAS,CAUCA,0,0,0,7,2,0,0,0.0
3546,2022.0,12.0,2022-12-26,AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA,PADILLA,CAUCA,0,0,0,0,0,0,0,0.0
3572,2022.0,11.0,2022-11-25,AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA,DEPARTAMENTO,VALLE DEL CAUCA,0,0,0,0,0,0,0,0.0


#### ***Arreglar formato en la fecha y reorganizar el orden de las columnas***

In [8]:
emergencias = emergencias.dropna()
emergencias = emergencias[emergencias != '']

if emergencias is not None:
    emergencias['AÑO'] = emergencias['AÑO'].astype(int, errors='ignore')
    emergencias['MES'] = emergencias['MES'].astype(int, errors='ignore')

# Eliminar la hora de la columna de fecha
emergencias['FECHA'] = pd.to_datetime(emergencias['FECHA'], format='ISO8601')
emergencias['FECHA'] = emergencias['FECHA'].dt.date

# Organizar los años y los meses de manera ascendente
emergencias = emergencias.sort_values(by=['AÑO', 'MES'], ascending=[True, True])

# Mapear los meses a su nombre
meses_map = {
    1: 'Enero',
    2: 'Febrero',
    3: 'Marzo',
    4: 'Abril',
    5: 'Mayo',
    6: 'Junio',
    7: 'Julio',
    8: 'Agosto',
    9: 'Septiembre',
    10: 'Octubre',
    11: 'Noviembre',
    12: 'Diciembre'
}

emergencias['MES_ORDEN'] = emergencias['MES']
emergencias['MES'] = emergencias['MES'].map(meses_map)

# Reordenar las columnas para que "MES_ORDEN" quede justo después de "MES"
columnas = list(emergencias.columns)

indice_mes = columnas.index('MES')

# Se remueve "MES_ORDEN" para evitar duplicados si ya existe
columnas.remove('MES_ORDEN')
columnas.insert(indice_mes + 1, 'MES_ORDEN')

# Reordena el DataFrame según la nueva lista de columnas
emergencias = emergencias[columnas]

#### ***Nos enfocaremos en los departamentos de la región Pacífico***

In [9]:
emergencias["DEPARTAMENTO"].value_counts()

DEPARTAMENTO
CUNDINAMARCA               5379
CAUCA                      3190
TOLIMA                     2750
NARIÑO                     2636
SANTANDER                  2532
ANTIOQUIA                  2498
HUILA                      2300
BOYACA                     2168
VALLE DEL CAUCA            1677
RISARALDA                  1465
CHOCO                      1453
CASANARE                   1201
NORTE DE SANTANDER         1097
MAGDALENA                  1029
BOLIVAR                     988
CESAR                       949
CORDOBA                     902
ATLANTICO                   891
META                        855
QUINDIO                     796
CALDAS                      775
SUCRE                       622
CAQUETA                     443
LA GUAJIRA                  426
PUTUMAYO                    409
ARAUCA                      326
BOGOTA, D.C.                265
VALLE                       199
VICHADA                     173
GUAVIARE                    137
GUAJIRA                    

In [10]:
diccionario_reemplazos = {
    "VALLE ": "VALLE DEL CAUCA",
    "VALLE": "VALLE DEL CAUCA"
}

emergencias["DEPARTAMENTO"] = emergencias["DEPARTAMENTO"].replace(diccionario_reemplazos)

departamentos_deseados = ["NARIÑO", "CAUCA", "VALLE DEL CAUCA", "CHOCO"]

emergencias = emergencias[emergencias["DEPARTAMENTO"].isin(departamentos_deseados)]

### ***Guardar como CSV***
---

In [11]:
emergencias.to_csv('../EmergenciasCC(2003-2023)_RegionPacifico.csv', index=False)